In [23]:
import numpy as np
import pandas as pd

a = np.array([[2., 1., 0., 5., 4.], [6., 10., 7., 1., 3.]])
df = pd.DataFrame(a, columns=['A', 'B', 'C', 'D', 'E'])

pd.concat((df,pd.DataFrame(df.columns[np.argsort(df.values)[:,-3:][:,::-1]], columns=['First','Second','Third'])), axis=1)



# pd.concat((df, pd.DataFrame(df.columns[np.argsort(df.values, axis=1)[:, -3:][:, ::-1]],columns=['First', 'Second', 'Third'])), axis=1)


,A,B,C,D,E,First,Second,Third
0,2.00000,1.00000,0.00000,5.00000,4.00000,D,E,A
1,6.00000,10.00000,7.00000,1.00000,3.00000,B,C,A


In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*
%matplotlib inline
#Imports
#dataprocessing & viturlization
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [7]:
sns.set(context='notebook',style="ticks",palette="GnBu_d",font_scale=1.5,font='ETBembo',rc={"figure.figsize": (10, 6)})
plt.rcParams['figure.figsize']=(15,10)
plt.show()
import warnings
warnings.filterwarnings('ignore') #为了整洁，去除弹出的warnings
pd.set_option('precision', 5) #设置精度
pd.set_option('display.float_format', lambda x: '%.5f' % x) #为了直观的显示数字，不采用科学计数法
pd.options.display.max_rows = 10 #最多显示200行

In [8]:
rdata = pd.read_csv('bank-full.csv',sep=';',dtype={'balance':np.float64})
radata = pd.read_csv('bank-additional-full.csv',sep=';',dtype={'balance':np.float64})

In [14]:
sorted(rdata.columns)

['age',
 'balance',
 'campaign',
 'contact',
 'day',
 'default',
 'duration',
 'education',
 'housing',
 'job',
 'loan',
 'marital',
 'month',
 'pdays',
 'poutcome',
 'previous',
 'y']

In [13]:
sorted(radata.columns)

['age',
 'campaign',
 'cons.conf.idx',
 'cons.price.idx',
 'contact',
 'day_of_week',
 'default',
 'duration',
 'education',
 'emp.var.rate',
 'euribor3m',
 'housing',
 'job',
 'loan',
 'marital',
 'month',
 'nr.employed',
 'pdays',
 'poutcome',
 'previous',
 'y']

In [26]:
##########################data_check#########################################
radata.columns
radata.info()
radata.describe()

radata.describe().astype(np.int64).T
radata.select_dtypes(include=['O']).describe().T.assign(missing_pct=radata.apply(lambda x : (len(x)-x.count())/len(x)))
(radata.select_dtypes(include=['float64']).describe().T) .drop(['25%','50%','75%'],axis=1)\
    .assign(missing_pct=radata.apply(lambda x: (len(x)-x.count())/len(x)),nunique = radata.apply(lambda x: x.nunique()),
            pct_10=radata.select_dtypes(include=['float64']).apply(lambda x: x.dropna().quantile(.1)),
            pct_25=radata.select_dtypes(include=['float64']).apply(lambda x: x.dropna().quantile(.25)),
            pct_50=radata.select_dtypes(include=['float64']).apply(lambda x: x.dropna().quantile(.5)),
            pct_75=radata.select_dtypes(include=['float64']).apply(lambda x: x.dropna().quantile(.75)),
            pct_90=radata.select_dtypes(include=['float64']).apply(lambda x: x.dropna().quantile(.9)))
demo_data = radata.loc[:,radata.columns.str.contains('')] #取包含‘num’的所有变量
sns.boxplot(data=demo_data,orient="h",color="c")

sns.despine(trim=True,offset=10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

ValueError: min() arg is an empty sequence

In [23]:
############################记录重复值检查与处理#############################
radata[radata.duplicated()==True].count()
radata[radata.duplicated()==True]
radata.drop_duplicates()

##########################空值检查与处理######################################
radata.isnull().any()
radata[radata.isnull().values==True]

age               False
job               False
marital           False
education         False
default           False
housing           False
loan              False
contact           False
month             False
day_of_week       False
duration          False
campaign          False
pdays             False
previous          False
poutcome          False
emp.var.rate      False
cons.price.idx    False
cons.conf.idx     False
euribor3m         False
nr.employed       False
y                 False
dtype: bool


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y


In [ ]:
#缺失值较多的特征处理
# 定义工资改变特征缺失值处理函数，将有变化设为Yes，缺失设为No
def set_salary_change(df):
    df.loc[(df.salary_change.notnull()), 'salary_change'] = 'Yes'
    df.loc[(df.salary_change.isnull(df)), 'salary_change'] = 'No'
    print df
    pd.df()
    print df
    return df
radata = set_salary_change(radata)

In [ ]:
#缺失值较少的特征处理
# 将所有行用各自的均值填充
radata.fillna(radata.mean())
# 也可以指定某些行进行填充
radata.fillna(radata.mean()['browse_his', 'card_num'])
# 用前一个数据代替NaN：method='pad'
radata.fillna(method='pad')
# 与pad相反，bfill表示用后一个数据代替NaN
radata.fillna(method='bfill')
# 插值法就是通过两点（x0，y0），（x1，y1）估计中间点的值
radata.interpolate()

In [ ]:
##
radata[radata.isnull()==True].count()
radata[radata.notnull()==True].count()

radata.fillna(0)
radata.dropna()
radata['loan_amnt']=radata['loan_amnt'].fillna(radata['total_pymnt']-radata['total_rec_int']).astype(np.int64)
radata['annual_inc']=radata['annual_inc'].fillna(radata['annual_inc'].mean())

##################异常值检查与处理###############################################
#连续变量
radata['age'].value_counts()
sns.distplot(radata['age'])
radata.boxplot(column='age')
radata.boxplot(column='age',by='education')
#分类变量
sns.stripplot(x='education',y='age',data=radata,jitter=True)
sns.stripplot(x='education',y='age', hue="y",data=radata, jitter=True)

#ata.replace([A, B], [A_R, B_R])  data.replace({A:A_R, B:B_R})
radata['loan_amnt'].replace([100000,36],radata['loan_amnt'].mean())
radata['age'].replace([158, 6], np.nan)
sigp = radata['age'].mean() + 3 * radata['age'].std()
sigm = radata['age'].mean() - 3 * radata['age'].std()
radata['age'].replace(radata[radata['age']>sigp].age,sigp)
radata['age'].replace(radata[radata['age']<sigm].age,sigm)

#检查和去字符空格
radata['education'].value_counts()
radata['term'] = radata['term'].map(str.strip)
radata['term'] = radata['term'].map(str.lstrip)
radata['term'] = radata['term'].map(str.rstrip)
#调大小写
radata['term']=radata['term'].map(str.upper)
radata['term']=radata['term'].map(str.lower)
radata['term']=radata['term'].map(str.title)
#检查每个变量类型是否一致
radata['emp_length'].apply(lambda x: x.isalpha())
radata['emp_length'].apply(lambda x: x. isalnum ())
radata['emp_length'].apply(lambda x: x. isdigit ())

##############修整数据格式#####################
radata['loan_amnt']=radata['loan_amnt'].astype(np.int64)
radata['issue_d']=pd.to_datetime(radata['issue_d'])
radata.dtypes

##############数据分箱######################
bins = [0, 5, 10, 15, 20]
group_names = ['A', 'B', 'C', 'D']
radata['categories'] = pd.cut(radata['open_acc'], bins, labels=group_names)

cutPoint=[0,30, 40, 50,80]
groupLabel=[0,1,2,3]
pd.cut(radata['age'],cutPoint)
radata['ageGroup'] =pd.cut(radata['age'],cutPoint,labels=groupLabel)
#按分位数
radata['ageGroup'] =pd.qcut(radata['age'],2)#分两组
pd.get_dummies(radata['ageGroup'] )
radata =pd.merge(radata, pd.get_dummies(radata['ageGroup'], prefix='age' ),right_index=True, left_index=True)
###########一个变量分裂############
grade_split = pd.DataFrame((x.split('-') for x in radata.grade),index=radata.index,columns=['grade','sub_grade'])
radata=pd.merge(radata,grade_split,right_index=True, left_index=True)